In [1]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/yaedin/twitter-disaster/main/raw/disaster_tweets.csv')
print(data.info())
print('----------------')
print(f"target column unique = {data.target.unique()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
None
----------------
target column unique = [1 0]


In [2]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)

In [3]:
train.drop('id', axis=1, inplace=True)
train.drop('keyword', axis=1, inplace=True)
train.drop('location', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)
test.drop('keyword', axis=1, inplace=True)
test.drop('location', axis=1, inplace=True)

print(train.info())
print(test.info())
print("train df:")
print(train['target'].value_counts(normalize=True) * 100)
print("test df:")
print(test['target'].value_counts(normalize=True) * 100)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6090 entries, 2762 to 4701
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    6090 non-null   object
 1   target  6090 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 142.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1523 entries, 54 to 1296
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1523 non-null   object
 1   target  1523 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 35.7+ KB
None
train df:
0    57.405583
1    42.594417
Name: target, dtype: float64
test df:
0    55.54826
1    44.45174
Name: target, dtype: float64


/home/ibrago/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [4]:
def clean_text(text):
  # get rid of all unwanted punctuation and digits
  text = text.replace("´", "'")
  digi_punct = "[^a-zA-Z' ]"
  text = re.sub(digi_punct, " ", text)
  text = " ".join(text.split())
  text = text.lower()
  return text
import re
train["text_clean"] = train["text"].apply(clean_text)
display(train.head())

/home/ibrago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,text,target,text_clean
2762,70 Years After Atomic Bombs Japan Still Strugg...,1,years after atomic bombs japan still struggles...
470,Suspect in latest US theatre attack had psycho...,1,suspect in latest us theatre attack had psycho...
6269,Is it bad to say that I'm kinda afraid of stor...,1,is it bad to say that i'm kinda afraid of stor...
964,å¤} New Ladies Shoulder Tote #Handbag Faux Lea...,0,new ladies shoulder tote handbag faux leather ...
2907,My parents don't believe in the dream. Sad.,0,my parents don't believe in the dream sad


In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
my_stopwords = stopwords.words("english")
train["wo_stopwords"] = train["text"].apply(
    lambda text: " ".join([word for word in text.split() if word not in my_stopwords])
)
train.head()

[nltk_data] Downloading package stopwords to /home/ibrago/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/ibrago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,text,target,text_clean,wo_stopwords
2762,70 Years After Atomic Bombs Japan Still Strugg...,1,years after atomic bombs japan still struggles...,70 Years After Atomic Bombs Japan Still Strugg...
470,Suspect in latest US theatre attack had psycho...,1,suspect in latest us theatre attack had psycho...,Suspect latest US theatre attack psychological...
6269,Is it bad to say that I'm kinda afraid of stor...,1,is it bad to say that i'm kinda afraid of stor...,Is bad say I'm kinda afraid storms storm???? help
964,å¤} New Ladies Shoulder Tote #Handbag Faux Lea...,0,new ladies shoulder tote handbag faux leather ...,å¤} New Ladies Shoulder Tote #Handbag Faux Lea...
2907,My parents don't believe in the dream. Sad.,0,my parents don't believe in the dream sad,My parents believe dream. Sad.


In [6]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
train["lemmatized"] = train["wo_stopwords"].apply(
  lambda text: " ".join([lemmatizer.lemmatize(word) for word in text.split()])
)

[nltk_data] Downloading package wordnet to /home/ibrago/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/home/ibrago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [7]:
test.head()

,text,target
54,Noches El-Bestia '@Alexis_Sanchez: happy to se...,0
6741,My room looks like a tornado passed through it...,0
567,@DetroitPls interested to see who will win thi...,0
1988,And here I was complaining about Phoenix Mode ...,0
2538,Winter Desolation of Death is also on Tumblr: ...,0


In [8]:
#for further use it makes sense to define train and test data for vectorization
X_train = train['lemmatized']
y_train = train['target']
X_test = test['text']
y_test = test['target']

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

pipeline1 = make_pipeline(CountVectorizer(), LogisticRegression())
pipeline2 = make_pipeline(TfidfVectorizer(), LogisticRegression())

print("CountVectorizer training data", cross_val_score(pipeline1, X_train, y_train, scoring='f1').mean())
print("TfidfVectorizer training data", cross_val_score(pipeline2, X_train, y_train, scoring='f1').mean())

CountVectorizer training data 0.7479187247369256
TfidfVectorizer training data 0.7332228807214597


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

pipeline1 = make_pipeline(CountVectorizer(), LogisticRegression())
pipeline2 = make_pipeline(TfidfVectorizer(), LogisticRegression())

print("CountVectorizer testing data", cross_val_score(pipeline1, X_test, y_test, scoring='f1').mean())
print("TfidfVectorizer testing data", cross_val_score(pipeline2, X_test, y_test, scoring='f1').mean())

CountVectorizer testing data 0.7113637015791398
TfidfVectorizer testing data 0.7022425843816874


In [11]:
#try with own text
list("I am a cow")

['I', ' ', 'a', 'm', ' ', 'a', ' ', 'c', 'o', 'w']

In [21]:
text = []
text.append("fire california")
text

['fire california']

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_fit = vectorizer.fit_transform(X_train)
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train_fit, y_train)
X_test_fit = vectorizer.transform(X_test)
textpred = vectorizer.transform(text)
y_pred = logreg.predict(textpred)
y_pred

array([1])

In [25]:
def test(tweet):
    text = []
    text.append(tweet)
    textpred = vectorizer.transform(text)
    y_pred = logreg.predict(textpred)
    print(y_pred)

In [34]:
test('in california is sunshine')

[1]


In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train_fit, y_train)

In [ ]:
X_test_fit = vectorizer.transform(X_test)
y_pred = logreg.predict(X_test_fit)

In [ ]:
model_score = logreg.score(X_train_fit, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy_percentage = 100 * accuracy
accuracy_percentage

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt="")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

pipeline1 = make_pipeline(CountVectorizer(), LogisticRegression())
pipeline2 = make_pipeline(TfidfVectorizer(), LogisticRegression())

print("CountVectorizer", cross_val_score(pipeline1, X_test, y_test).mean())
print("TfidfVectorizer", cross_val_score(pipeline2, X_test, y_test).mean())

In [ ]:
model.predict